# Car Dashboard
This notebook contains data of used cars from 1908-2019. We fill clean the data and make some histograms and a scatterplot to deploy an app on Render. 

In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px

2023-01-12 21:29:12.040 INFO    numexpr.utils: NumExpr defaulting to 4 threads.


In [2]:
df = pd.read_csv(r'C:\Users\tomin\Documents\Practicum Projects\Sprint 4\Car-Dashboard\vehicles_us.csv')
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


There are some columns with missing values such as model_year, cylinders, odometer, paint_color, and is_4wd. 

In [4]:
#filling in missing values in model_year with median based on car model 
df['model_year'] = df['model_year'].fillna(df.groupby(['model'])['model_year'].transform('median'))

In [5]:
#filling in 0 for those without 4wd
df.is_4wd.fillna(0, inplace=True)

In [6]:
#filling in missing values in cylinders based on car model 
df['cylinders'] = df['cylinders'].fillna(df.groupby(['model'])['cylinders'].transform('median'))

In [7]:
#filing in missing values in odometer based on model and model_year
df['odometer'] = df['odometer'].fillna(df.groupby(['model','model_year'])['odometer'].transform('median'))

In [8]:
#filling in the missing values with unknown for paint_color 
df.paint_color.fillna('unknown', inplace=True)

In [9]:
#looking to make sure there are no missing values before proceeding 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    51525 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     51525 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      51442 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   51525 non-null  object 
 10  is_4wd        51525 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [10]:
#extracting the names of the manufacterer
split = df['model'].str.split(pat=' ', n=1, expand=True)

In [11]:
#adding the manufacterer and model names into new columns, dropping the one with both 
df['manufacterer'] = split[0]
df['model_name'] = split[1]
df.drop('model', axis=1, inplace=True)
df.head()

,price,model_year,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,manufacterer,model_name
0,9400,2011.0,good,6.0,gas,145000.0,automatic,SUV,unknown,1.0,2018-06-23,19,bmw,x5
1,25500,2011.0,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50,ford,f-150
2,5500,2013.0,like new,4.0,gas,110000.0,automatic,sedan,red,0.0,2019-02-07,79,hyundai,sonata
3,1500,2003.0,fair,8.0,gas,177500.0,automatic,pickup,unknown,0.0,2019-03-22,9,ford,f-150
4,14900,2017.0,excellent,4.0,gas,80903.0,automatic,sedan,black,0.0,2019-04-02,28,chrysler,200


## Plotly Dashboard

In [12]:
#creating header 
st.header('Used Cars from, 1908-2019')
st.write("""
##### The data below shows the type of cars from each manufacterer from 1908-2019
""")
#let users decide whether they want to see the odometer or not using a checkbox
exclude_odometer = st.checkbox('Exclude odometer reading')
if exclude_odometer:
    df = df.drop('odometer', axis=1)
#inserting the dataframe
st.dataframe(df)

2023-01-12 21:29:14.784 
  command:

    streamlit run C:\Users\tomin\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [13]:
st.header('Number of Vehicle Types by Manufacterer')
#creating a plotly histogram
fig1 = px.histogram(df, x='manufacterer', color='type')
#displaying the figure with streamlit
st.write(fig1)

In [14]:
st.header('Price Range of Vehicles Per Model Year')
#creating a scatterplot
fig2 = px.scatter(df, x='model_year', y='price')
#displaying the scatterplot
st.write(fig2)

In [15]:
st.header('Overall Transmission Type of Vehicles')
st.write("""
##### The data below that the majority of cars listed are automatic transmission.
""")
#making a histogram
fig3 = px.histogram(df, x='transmission')
#displaying the histogram
st.write(fig3)

In [16]:
st.header('Overall Types of Cars Listed')
#creating a scatterplot
fig4 = px.histogram(df, x='type')
#displaying the scatterplot
st.write(fig4)

### Conclusion:
We have managed to clean the data and take a look at the number of vehicles per manufacterer, the overall price range for all the vehicles listed, the transmission type of the vehicles, and types of vehicles listed in this data. 